Note: requires tesseract (impira/layoutlm-document-qa) & poppler (pdf2image) to be installed

In [1]:
import os
data = "../../data/datasets/pdf_tressl"
os.listdir(data)[0]

PDF_path = f"{data}/{os.listdir(data)[0]}"

PDF_path

'../../data/datasets/pdf_tressl/AddendumDetail-Report-202311230750 (2023-11-23).pdf'

Seperate each page

In [2]:
import PyPDF2
import os

from marker.convert import convert_single_pdf
from marker.models import load_all_models
import json

fname = PDF_path
model_lst = load_all_models()

print("models loaded")

from vector_store import VectorStore

vs = VectorStore()

def marker(pdf_path):
    full_text, out_meta = convert_single_pdf(pdf_path, model_lst, max_pages=None, parallel_factor=2)

    out_path = f'{pdf_path}_marked.md'

    with open(out_path, "w+", encoding='utf-8') as f:
        f.write(full_text)

    out_meta_filename = out_path.rsplit(".", 1)[0] + "_meta.json"
    with open(out_meta_filename, "w+") as f:
        f.write(json.dumps(out_meta, indent=4))

    return full_text

def split_pdf_into_pages(pdf_path, output_folder):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for page_num in range(len(reader.pages)):
            writer = PyPDF2.PdfWriter()
            writer.add_page(reader.pages[page_num])

            output_filename = os.path.join(output_folder, f'page_{page_num + 1}.pdf')
            with open(output_filename, 'wb') as output_file:
                writer.write(output_file)

            marked_text = marker(output_filename)
            vs.store_page(marked_text, page_num + 1)
            print(f"Ingested: {output_filename}")

split_pdf_into_pages(PDF_path, 'temp')


ModuleNotFoundError: No module named 'marker.convert'

In [ ]:
from pdf2image import convert_from_path
from PIL import Image

In [ ]:
def pdf_to_image(pdf_path):
    images = convert_from_path(pdf_path)

    # Calculate the total height of all images (assuming they have the same width)
    total_height = sum(img.size[1] for img in images)
    max_width = max(img.size[0] for img in images)

    # Create a new image with the appropriate height and width
    combined_image = Image.new('RGB', (max_width, total_height))

    # Paste the images together
    y_offset = 0
    for img in images:
        combined_image.paste(img, (0, y_offset))
        y_offset += img.size[1]

    return combined_image

In [ ]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

# nlp = pipeline(
#     "document-question-answering", 
#     model="impira/layoutlm-invoices"
# )

# nlp = pipeline("document-question-answering", model="naver-clova-ix/donut-base-finetuned-docvqa")

query = "What is the length used by BRIDGE (LOC 131363) - S00132?"
res_obj = vs.query(query)

res_obj[0][0]

7

In [ ]:
img = pdf_to_image(f"temp/page_{res_obj[0][0]}.pdf")

ans = nlp(
    img,
    query
)

In [ ]:
print(ans[0]['answer'])

0.01 km


In [ ]:
nlp("mistral_paper.png", "What is the vocab_size?")

[{'score': 0.9039022922515869, 'answer': '32000', 'start': 427, 'end': 427}]